<a href="https://colab.research.google.com/github/aryan-at-ul/Med-MNISt-classification-GCNN/blob/main/gcnn_benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install medmnist

In [1]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms

import medmnist
from medmnist import INFO, Evaluator

In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [3]:
import gc

data_flag = 'pathmnist'
# data_flag = 'breastmnist'
download = True

NUM_EPOCHS = 3
BATCH_SIZE = 64
lr = 0.001

info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

DataClass = getattr(medmnist, info['python_class'])

In [4]:
# preprocessing
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Grayscale(num_output_channels=1),
    transforms.Normalize(mean=[.5], std=[.5])
])
BATCH_SIZE = 64
# load the data
train_dataset = DataClass(split='train', transform=data_transform, download=download)
test_dataset = DataClass(split='test', transform=data_transform, download=download)

train_dataset = [(one[0],one[1][0],) for one in train_dataset]
test_dataset = [(one[0],one[1][0],) for one in test_dataset]

train_dataset = train_dataset[0:89984]
test_dataset = test_dataset[0:7168]

pil_dataset = DataClass(split='train', download=download)

# encapsulate data into dataloader form
train_loader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
train_loader_at_eval = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = data.DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

Using downloaded and verified file: /root/.medmnist/pathmnist.npz
Using downloaded and verified file: /root/.medmnist/pathmnist.npz
Using downloaded and verified file: /root/.medmnist/pathmnist.npz


In [5]:
# train_loader = []
# y = torch.Tensor(torch.flatten(y))
print(len(test_dataset))
print(len(train_dataset))


for i, (x, y) in enumerate(train_loader):
        print(x.shape, y.shape) 
        break
# for one in train_dataset:
#     # for x in one:
#     #     print(x.shape)
#     #     break
#     # break
#     print(one[1][0]) 
#     break


7168
89984
torch.Size([64, 1, 28, 28]) torch.Size([64])


In [6]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
from scipy.spatial.distance import cdist


import torch
import torch.nn.functional as F
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp

from torch_geometric.utils import grid
from torch_geometric.nn import SplineConv,GraphConv,TopKPooling

In [7]:
def to_batch(edge_index, pos, batch_size):
    edge_indices = [edge_index + pos.size(0) * i for i in range(batch_size)]
    edge_index = torch.cat(edge_indices, dim=1)
    pos = torch.cat([pos] * batch_size, dim=0)
    edge_attr = pos[edge_index[0]] - pos[edge_index[1]]
    return edge_index, (edge_attr + 1.) / 2.


edge_index1, edge_attr1 = to_batch(*grid(28, 28), batch_size=64)
edge_index2, edge_attr2 = to_batch(*grid(14, 14), batch_size=64)


In [8]:
# # !pip uninstall torch-scatter torch-sparse torch-cluster torch-spline-conv & y
# import torch

# def format_pytorch_version(version):
#   return version.split('+')[0]

# TORCH_version = torch.__version__
# TORCH = format_pytorch_version(TORCH_version)

# def format_cuda_version(version):
#   return 'cu' + version.replace('.', '')

# CUDA_version = torch.version.cuda
# CUDA = format_cuda_version(CUDA_version)

# !pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
# !pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
# !pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
# !pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html

In [8]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = SplineConv(1, 32, dim=2, kernel_size=3)
        self.conv2 = SplineConv(32, 64, dim=2, kernel_size=3)
        self.fc1 = torch.nn.Linear(3136, 512)
        self.fc2 = torch.nn.Linear(512, 9)

    def forward(self, x):
        x = x.view(-1, 1)
        x = F.elu(self.conv1(x.view(-1, 1), edge_index1, edge_attr1))
        x = x.view(64, 28, 28, 32).permute(0, 3, 1, 2)
        x = F.max_pool2d(x, kernel_size=2)
        x = x.permute(0, 2, 3, 1).contiguous().view(-1, 32)
        x = F.elu(self.conv2(x, edge_index2, edge_attr2))
        x = x.view(64, 14, 14, 64).permute(0, 3, 1, 2)
        x = F.max_pool2d(x, kernel_size=2)
        x = x.contiguous().view(64, -1)

        x = F.elu(self.fc1(x))
        return F.log_softmax(self.fc2(x), dim=1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
edge_index1, edge_attr1 = edge_index1.to(device), edge_attr1.to(device)
edge_index2, edge_attr2 = edge_index2.to(device), edge_attr2.to(device)


In [9]:
def train(epoch):
    model.train()

    for i, (x, y) in enumerate(train_loader):
        # print("y.shape",y.shape)
        y = torch.Tensor(y)
        # print("y.shape after flattening",y.shape)
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        pred = model(x)
        # print(y)
        # print("pred shape",pred.shape)
        loss = F.nll_loss(model(x), y)
        loss.backward()
        optimizer.step()
        # print(i, len(train_loader), loss.item())

def test():
    model.eval()

    correct = 0
    for i, (x, y) in enumerate(test_loader):
        # y = torch.Tensor(torch.flatten(y))
        y = torch.Tensor(y)
        x, y = x.to(device), y.to(device)
        pred = model(x).max(1)[1]
        # print(pred.shape,"jjdhfjhsdjfhsjdhfjksdhfjhsd")
        # print(y.shape,"kdjhsfkshdkfjhkjsdhfkshdkjhsfkdhskj")
        correct += pred.eq(y).sum().item()
    return correct / len(test_dataset)



In [15]:
for epoch in range(1,3):
    train(epoch)
    test_acc = test()
    print('Epoch: {:02d}, Test: {:.4f}'.format(epoch, test_acc))

Epoch: 01, Test: 0.5960
Epoch: 02, Test: 0.6875


In [13]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GraphConv(64, 128)
        self.pool1 = TopKPooling(128, ratio=0.8)
        self.conv2 = GraphConv(128, 128)
        self.pool2 = TopKPooling(128, ratio=0.8)
        self.conv3 = GraphConv(128, 128)
        self.pool3 = TopKPooling(128, ratio=0.8)
        
        self.lin1 = torch.nn.Linear(256, 128)
        self.lin2 = torch.nn.Linear(128, 64)
        self.lin3 = torch.nn.Linear(64, 9)
        
    def forward(self, data):
            x, edge_index, batch = data.x, data.edge_index, data.batch
            x = F.relu(self.conv1(x, edge_index))
            x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, None, batch)
            x1 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)
            x = F.relu(self.conv2(x, edge_index))
            x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, None, batch)
            x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)
            x = F.relu(self.conv3(x, edge_index))
            x, edge_index, _, batch, _, _ = self.pool3(x, edge_index, None, batch)
            x3 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)
            x = x1 + x2 + x3
            x = F.relu(self.lin1(x))
            x = F.dropout(x, p=0.5, training=self.training)
            x = F.relu(self.lin2(x))
            x = F.log_softmax(self.lin3(x), dim=-1)
    
            return x
      

In [ ]:
# def train(epoch):
#     model.train()

#     loss_all = 0
#     for data in train_loader:
#         data = data.to(device)
#         optimizer.zero_grad()
#         output = model(data)
#         loss = F.nll_loss(output, data.y)
#         loss.backward()
#         loss_all += data.num_graphs * loss.item()
#         optimizer.step()
#     return loss_all / len(train_dataset)

# def test(loader):
#     model.eval()
#     correct = 0
#     for data in loader:
#         data = data.to(device)
#         pred = model(data).max(dim=1)[1]
#         correct += pred.eq(data.y).sum().item()
#     return correct / len(loader.dataset)


def train(epoch):
    model.train()

    for i, (x, y) in enumerate(train_loader):
        # print("y.shape",y.shape)
        y = torch.Tensor(y)
        # print("y.shape after flattening",y.shape)
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        pred = model(x)
        # print(y)
        # print("pred shape",pred.shape)
        loss = F.nll_loss(model(x), y)
        loss.backward()
        optimizer.step()
        # print(i, len(train_loader), loss.item())

def test():
    model.eval()

    correct = 0
    for i, (x, y) in enumerate(test_loader):
        # y = torch.Tensor(torch.flatten(y))
        y = torch.Tensor(y)
        x, y = x.to(device), y.to(device)
        pred = model(x).max(1)[1]
        # print(pred.shape,"jjdhfjhsdjfhsjdhfjksdhfjhsd")
        # print(y.shape,"kdjhsfkshdkfjhkjsdhfkshdkjhsfkdhskj")
        correct += pred.eq(y).sum().item()
    return correct / len(test_dataset)



for epoch in range(1,100):
    train(epoch)
    test_acc = test()
    print('Epoch: {:02d}, Test: {:.4f}'.format(epoch, test_acc))

Epoch: 01, Test: 0.7194


In [13]:
# #mnist code from tutorials

# import torch
# import torch.nn.functional as F
# from torchvision.datasets import MNIST
# from torchvision.transforms import ToTensor
# from torch.utils.data import DataLoader

# from torch_geometric.utils import grid
# from torch_geometric.nn import SplineConv

# train_dataset = MNIST('/tmp/MNIST', train=True, transform=ToTensor(),download=True)
# test_dataset = MNIST('/tmp/MNIST', train=False, transform=ToTensor(),download=True)
# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True,
#                           drop_last=True)
# test_loader = DataLoader(test_dataset, batch_size=64, drop_last=True)


# def to_batch(edge_index, pos, batch_size):
#     edge_indices = [edge_index + pos.size(0) * i for i in range(batch_size)]
#     edge_index = torch.cat(edge_indices, dim=1)
#     pos = torch.cat([pos] * batch_size, dim=0)
#     edge_attr = pos[edge_index[0]] - pos[edge_index[1]]
#     return edge_index, (edge_attr + 1.) / 2.


# edge_index1, edge_attr1 = to_batch(*grid(28, 28), batch_size=64)
# edge_index2, edge_attr2 = to_batch(*grid(14, 14), batch_size=64)


# class Net(torch.nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = SplineConv(1, 32, dim=2, kernel_size=3)
#         self.conv2 = SplineConv(32, 64, dim=2, kernel_size=3)
#         self.fc1 = torch.nn.Linear(3136, 512)
#         self.fc2 = torch.nn.Linear(512, 10)

#     def forward(self, x):
#         x = x.view(-1, 1)
#         x = F.elu(self.conv1(x.view(-1, 1), edge_index1, edge_attr1))
#         x = x.view(64, 28, 28, 32).permute(0, 3, 1, 2)
#         x = F.max_pool2d(x, kernel_size=2)
#         x = x.permute(0, 2, 3, 1).contiguous().view(-1, 32)
#         x = F.elu(self.conv2(x, edge_index2, edge_attr2))
#         x = x.view(64, 14, 14, 64).permute(0, 3, 1, 2)
#         x = F.max_pool2d(x, kernel_size=2)
#         x = x.contiguous().view(64, -1)

#         x = F.elu(self.fc1(x))
#         return F.log_softmax(self.fc2(x), dim=1)


# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = Net().to(device)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# edge_index1, edge_attr1 = edge_index1.to(device), edge_attr1.to(device)
# edge_index2, edge_attr2 = edge_index2.to(device), edge_attr2.to(device)

In [12]:
print(sum(p.numel() for p in model.parameters())) 

# for i, (x, y) in enumerate(train_loader):
#         print(x.shape, y.shape) 
#         break

1631657


In [ ]:
# def train(epoch):
#     model.train()

#     for i, (x, y) in enumerate(train_loader):
#         print("y.shape",y.shape)
#         x, y = x.to(device), y.to(device)
#         optimizer.zero_grad()
#         pred = model(x)
#         print(y)
#         print("pred.shape",pred.shape)
#         loss = F.nll_loss(pred, y)
#         loss.backward()
#         optimizer.step()
#         print(i, len(train_loader), loss.item())


# def test():
#     model.eval()

#     correct = 0
#     for i, (x, y) in enumerate(test_loader):
#         x, y = x.to(device), y.to(device)
#         pred = model(x).max(1)[1]
#         correct += pred.eq(y).sum().item()
#     return correct / len(test_dataset)


# for epoch in range(1, 2):
#     train(epoch)
#     test_acc = test()
#     print('Epoch: {:02d}, Test: {:.4f}'.format(epoch, test_acc))